In [27]:
!python checkpoint_trainer.py --niters 1 --batch-size 20 --dataset sgrA --norm --shuffle --sample-tp 0.1 --mse-weight 1.0 --imab-dim 64 --cab-dim 256 --decoder-dim 128 --nlayers 1 --sample-type random --num-ref-points 128

checkpoint_trainer.py --niters 1 --batch-size 20 --dataset sgrA --norm --shuffle --sample-tp 0.1 --mse-weight 1.0 --imab-dim 64 --cab-dim 256 --decoder-dim 128 --nlayers 1 --sample-type random --num-ref-points 128
(6300, 960, 9) (1260, 960, 9) (840, 960, 9)
/home/gsasseville/.local/share/virtualenvs/Tripletformer-nUF7tw2u/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
No checkpoint file found. Training from Epoch 1
Experiment ID 4248493
torch.Size([20, 3840, 2])
tensor([[[0., 1.],
         [1., 1.],
         [2., 1.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 1.],
         [2., 1.],
         [3., 1.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 1.],
         [2., 1.],
         [0., 1.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        .

In [32]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import utils
import layers
import pdb
r = np.array([[1,2,3], [4,5,6]])
m = np.array([[True, False, True], [False, False, True]])
r[m]

full_len = 2*4 # Timesteps * dim. 960*4
pad = lambda v: F.pad(v, [0, full_len - len(v)], value=0)
C = torch.ones((1, 2, 4), dtype=torch.int64).cumsum(-1) - 1
mk = torch.ones(C.size())
mk[0][0][1] = 0
mk[0][0][2] = 0
mk[0][1][0] = 0
mk[0][1][3] = 0
mk_bool = mk.to(bool)
print('Original C is:\n', C)
print('mk_bool is:\n', mk_bool)
C = torch.stack([pad(r[m]) for r, m in zip(C, mk_bool)]).contiguous()
mk = torch.stack([pad(r[m]) for r, m in zip(mk, mk_bool)]).contiguous()
C_ = C

print('C after Padding is:\n', C)

C = torch.nn.functional.one_hot(C, num_classes =4)

print('C after one-hot is:\n', C)

target_context = torch.cat([tau[:,:,None], C], -1).contiguous()
target_mask = torch.stack([C_, mk], -1)
obs_len = torch.max(target_mask[:,:,1].sum(-1)).to(torch.int64) # number of points to predict

print(target_mask)

print(target_mask[:, :obs_len])

Original C is:
 tensor([[[0, 1, 2, 3],
         [0, 1, 2, 3]]])
mk_bool is:
 tensor([[[ True, False, False,  True],
         [False,  True,  True, False]]])
C after Padding is:
 tensor([[0, 3, 1, 2, 0, 0, 0, 0]])
C after one-hot is:
 tensor([[[1, 0, 0, 0],
         [0, 0, 0, 1],
         [0, 1, 0, 0],
         [0, 0, 1, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0],
         [1, 0, 0, 0]]])
tensor([[[0., 1.],
         [3., 1.],
         [1., 1.],
         [2., 1.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]])
tensor([[[0., 1.],
         [3., 1.],
         [1., 1.],
         [2., 1.]]])
